In [1]:
import pandas as pd
import numpy as np
# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [22]:
file_name = 'df_all_update.csv' ######### change file name here

df = pd.read_csv(file_name)

df_message = df[['time_text','author.name','author.id', 'message','trans_eng']]
df_message.rename(columns={'trans_eng':'messages_eng',}, inplace=True)
df_message.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100523 entries, 0 to 100522
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   time_text     100523 non-null  object
 1   author.name   100504 non-null  object
 2   author.id     100523 non-null  object
 3   message       100522 non-null  object
 4   messages_eng  100523 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB


In [23]:
# drop rows with null values
df_message = df_message.dropna()
df_message.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100503 entries, 0 to 100522
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   time_text     100503 non-null  object
 1   author.name   100503 non-null  object
 2   author.id     100503 non-null  object
 3   message       100503 non-null  object
 4   messages_eng  100503 non-null  object
dtypes: object(5)
memory usage: 4.6+ MB


In [24]:
# convert emoji to text 
import emoji
def convert_emoji_to_text(text):
    return emoji.demojize(text)

df_message['message_emo'] = df_message['message'].apply(convert_emoji_to_text)
df_message.head()

,time_text,author.name,author.id,message,messages_eng,message_emo
0,-0:24,Venancio Reyna,UCeSuNe9wK0n2bahv0X6PkZQ,México :sparkles::heart_on_fire:,México :sparkles::heart_on_fire:,México :sparkles::heart_on_fire:
1,-0:23,Ploylymumu,UCos_UgTKQQZEUXLbbl2Y9dg,🇹🇭:blue_heart:,🇹🇭:blue_heart:,:Thailand::blue_heart:
2,-0:23,Oemah Printing,UC1XCKk9xzLwUMgY8f7QsdYQ,indonesia,Indonesia,indonesia
3,-0:23,Tiên Hoàng,UCnXDXORGJCjrgsa7TkikVEw,VIỆT NAM 🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳,Vietnam 🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳,VIỆT NAM :Vietnam::Vietnam::Vietnam::Vietnam::...
4,-0:21,TRUONG NGUYEN,UCYVwchNJN8uZbD7SWFGaL5g,VIỆT NAM 🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳,Vietnam 🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳,VIỆT NAM :Vietnam::Vietnam::Vietnam::Vietnam::...


In [30]:
###### 3.1 extract from name ######
import pycountry
def extract_country(text):
    for country in pycountry.countries:
        # if country.name or country.alpha_2 or country.alpha_3 or country.flag in text:
        if country.name.lower() in text.lower():
            return country.name   
    return 'N/A'

###### 3.2 country_nospace ######
def extract_country_nospace(text):
    for country in pycountry.countries:
        # if country.name or country.alpha_2 or country.alpha_3 or country.flag in text:
        if country.name.lower().replace(' ', '') in text.lower():
            return country.name   
    return 'N/A'

###### 3.3 flag ######
def extract_flag(text):
    for country in pycountry.countries:
        # if country.name or country.alpha_2 or country.alpha_3 or country.flag in text:
        if country.flag in text:
            return country.name        
    return 'N/A'


###### 3.5 regex ######
import re
import pycountry

def extract_country_re(text):
    # Get a list of all country names
    countries = [country.name.lower() for country in pycountry.countries]
    # Join the country names into a single string with '|' separators
    pattern = '|'.join(countries)
    # Compile the regular expression pattern
    regex = re.compile(pattern)
    # Find the first match in the text
    match = regex.search(text.lower())
    # Return the matched country name, or 'N/A' if no match was found
    return match.group() if match else 'N/A'

def extract_country_re2(text):
    # Get a list of all country names and common abbreviations
    countries = [country.name for country in pycountry.countries]
    countries += [country.alpha_2 for country in pycountry.countries]
    countries += [country.alpha_3 for country in pycountry.countries]   
    # Join the country names and abbreviations into a single string with '|' separators
    pattern = '|'.join(countries)  
    # Add regex pattern to match variations in country names
    # (e.g. 'United States' or 'United St.' or 'U.S.' or 'USA')
    # Allow for optional spaces between words
    pattern = r'\b(?:{})\b'.format(pattern)
    pattern = pattern.replace(' ', r'\s*') 
    # Compile the regular expression pattern
    regex = re.compile(pattern, re.IGNORECASE)
    # Find the first match in the text
    match = regex.search(text)
    # Return the matched country name, or 'N/A' if no match was found
    return match.group() if match else 'N/A'

def extract_country_re3(text):
    # Get a list of all country names and common abbreviations
    countries = [country.name.lower() for country in pycountry.countries]
    countries += [country.name.lower().replace(' ', '') for country in pycountry.countries]
    countries += [country.alpha_2.lower() for country in pycountry.countries]
    countries += [country.alpha_3.lower() for country in pycountry.countries]
    # Join the country names and abbreviations into a single string with '|' separators
    pattern = '|'.join(countries)

    # Add regex pattern to match variations in country names
    # (e.g. 'United States' or 'United St.' or 'U.S.' or 'USA')
    # Allow for optional spaces between words
    pattern = r'\b(?:{})\b'.format(pattern)
    pattern = pattern.replace(' ', r'\s*')
    # Compile the regular expression pattern
    regex = re.compile(pattern, re.IGNORECASE)
    # Find all matches in the text
    matches = regex.findall(text)
    # Return the list of matches, or an empty list if no matches were found
    return matches

###### 3.6 NLTK ######
import nltk
def extract_country_nltk(text):
    try:
        # Tokenize the text
        tokens = nltk.word_tokenize(text)
        # Part-of-speech tag the tokens
        pos_tags = nltk.pos_tag(tokens)
        # Perform named entity recognition
        entities = nltk.ne_chunk(pos_tags)
        # Extract the location entities from the named entity tree
        locations = [e[0] for e in entities if isinstance(e, nltk.tree.Tree) and e.label() == 'GPE']
        # # Return the list of locations, or an empty list if no locations were found
        return locations
    except:
        return "error"

def extract_country_nltk_2(text):
    for i in enumerate(text):
        ii = i[1][0].lower()
        for country in pycountry.countries:
            if ii in country.name.lower():
                return country.name
            elif ii in country.name.lower().replace(' ', ''):
                return country.name
    return 'N/A'

def extract_country_nltk_3(text):    
    # Get a list of all country names and common abbreviations
    countries = [country.name.lower() for country in pycountry.countries]
    countries += [country.name.lower().replace(' ', '') for country in pycountry.countries]
    countries += [country.alpha_2.lower() for country in pycountry.countries]
    countries += [country.alpha_3.lower() for country in pycountry.countries]
   
    # Join the country names and abbreviations into a single string with '|' separators
    pattern = '|'.join(countries) 
    # Add regex pattern to match variations in country names
    # (e.g. 'United States' or 'United St.' or 'U.S.' or 'USA')
    # Allow for optional spaces between words
    pattern = r'\b(?:{})\b'.format(pattern)
    pattern = pattern.replace(' ', r'\s*')
    for i in enumerate(text):
        ii = i[1][0].lower()
        if ii in pattern:
            return ii
    return 'N/A'

In [26]:
# Step 3.1: extract country from name
df_message['pycountry_name'] = df_message['messages_eng'].apply(extract_country)

In [27]:
# Step 3.2: extract country from country_nospace
df_message['pycountry_nospace'] = df_message['messages_eng'].apply(extract_country_nospace)

In [28]:
# Step 3.3: extract country from flag
df_message['pycountry_flag'] = df_message['message'].apply(extract_flag)

In [32]:
# Step 3.5: extract country from regex
df_message['country_re'] = df_message['messages_eng'].apply(extract_country_re)
df_message['country_re2'] = df_message['messages_eng'].apply(extract_country_re2)
df_message['country_re3'] = df_message['messages_eng'].apply(extract_country_re2)

In [33]:
# Step 3.6: extract country from nltk
df_message['country_nltk'] = df_message['messages_eng'].apply(extract_country_nltk)
df_message['country_nltk_2'] = df_message['country_nltk'].apply(extract_country_nltk_2)
df_message['country_nltk_3'] = df_message['country_nltk'].apply(extract_country_nltk_3)

In [35]:
df_message.columns

Index(['time_text', 'author.name', 'author.id', 'message', 'messages_eng',
       'message_emo', 'pycountry_name', 'pycountry_nospace', 'pycountry_flag',
       'country_re', 'country_re2', 'country_re3', 'country_nltk',
       'country_nltk_2', 'country_nltk_3'],
      dtype='object')

In [38]:
df_country = df_message[['time_text', 'author.name','message' ,'pycountry_name', 'pycountry_nospace', 'pycountry_flag', 'country_re', 'country_re2', 'country_re3', 'country_nltk_2', 'country_nltk_3']]
df_country.head()

,time_text,author.name,message,pycountry_name,pycountry_nospace,pycountry_flag,country_re,country_re2,country_re3,country_nltk_2,country_nltk_3
0,-0:24,Venancio Reyna,México :sparkles::heart_on_fire:,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,-0:23,Ploylymumu,🇹🇭:blue_heart:,N/A,N/A,Thailand,N/A,N/A,N/A,N/A,N/A
2,-0:23,Oemah Printing,indonesia,Indonesia,Indonesia,N/A,indonesia,Indonesia,Indonesia,Indonesia,indonesia
3,-0:23,Tiên Hoàng,VIỆT NAM 🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳,N/A,Viet Nam,Viet Nam,N/A,Vietnam,Vietnam,Viet Nam,vietnam
4,-0:21,TRUONG NGUYEN,VIỆT NAM 🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳,N/A,Viet Nam,Viet Nam,N/A,Vietnam,Vietnam,Viet Nam,vietnam


In [45]:
# change to lower case
df_country['pycountry_name'] = df_country['pycountry_name'].str.lower()
df_country['pycountry_nospace'] = df_country['pycountry_nospace'].str.lower()
df_country['pycountry_flag'] = df_country['pycountry_flag'].str.lower()
df_country['country_re'] = df_country['country_re'].str.lower()
df_country['country_re2'] = df_country['country_re2'].str.lower()
df_country['country_re3'] = df_country['country_re3'].str.lower()
df_country['country_nltk_2'] = df_country['country_nltk_2'].str.lower()
df_country['country_nltk_3'] = df_country['country_nltk_3'].str.lower()

In [46]:
# vote for the country
df_country['country_vote'] = df_country[['pycountry_name', 'pycountry_nospace', 'pycountry_flag', 'country_re', 'country_re2', 'country_re3', 'country_nltk_2', 'country_nltk_3']].mode(axis=1)[0]
df_country

,time_text,author.name,message,pycountry_name,pycountry_nospace,pycountry_flag,country_re,country_re2,country_re3,country_nltk_2,country_nltk_3,country_vote
0,-0:24,Venancio Reyna,México :sparkles::heart_on_fire:,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a
1,-0:23,Ploylymumu,🇹🇭:blue_heart:,n/a,n/a,thailand,n/a,n/a,n/a,n/a,n/a,n/a
2,-0:23,Oemah Printing,indonesia,indonesia,indonesia,n/a,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia
3,-0:23,Tiên Hoàng,VIỆT NAM 🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳,n/a,viet nam,viet nam,n/a,vietnam,vietnam,viet nam,vietnam,viet nam
4,-0:21,TRUONG NGUYEN,VIỆT NAM 🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳🇻🇳,n/a,viet nam,viet nam,n/a,vietnam,vietnam,viet nam,vietnam,viet nam
...,...,...,...,...,...,...,...,...,...,...,...,...
100518,3:47:41,Nusaba Monhinlad,อิงฟ้ารอง 1 ค่ะ,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a
100519,3:47:41,Pitan Danlammajak,เก่งที่สุดแล้วไทยแลนด์,thailand,thailand,n/a,thailand,thailand,thailand,thailand,thailand,thailand
100520,3:47:42,Sorayut Hitjamnong,พี่ฟ้าเก่งที่สุดด สมมง:crown:🇧🇷 เลิฟฟ,n/a,n/a,brazil,n/a,is,is,n/a,n/a,n/a
100521,3:47:42,faii jilada,สมแล้ววว พี่อิงมาไกลสุดๆเลยยยย,n/a,n/a,n/a,n/a,so,so,n/a,n/a,n/a


In [48]:
df_country.country_vote.value_counts()

n/a          69292
thailand     12476
cambodia      5266
brazil        4360
indonesia     3723
             ...  
mali             1
turkey           1
che              1
oman             1
czechia          1
Name: country_vote, Length: 70, dtype: int64

In [56]:
# group by country
df_country_group = df_country.groupby(['country_vote']).count()
df_country_group

,time_text,author.name,message,pycountry_name,pycountry_nospace,pycountry_flag,country_re,country_re2,country_re3,country_nltk_2,country_nltk_3
country_vote,,,,,,,,,,,
ai,1,1,1,1,1,1,1,1,1,1,1
angola,10,10,10,10,10,10,10,10,10,10,10
argentina,3,3,3,3,3,3,3,3,3,3,3
australia,10,10,10,10,10,10,10,10,10,10,10
bangladesh,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...
ukraine,24,24,24,24,24,24,24,24,24,24,24
united kingdom,2,2,2,2,2,2,2,2,2,2,2
viet nam,1398,1398,1398,1398,1398,1398,1398,1398,1398,1398,1398


In [57]:
# view all countries
df_country_group.index.tolist()

['ai',
 'angola',
 'argentina',
 'australia',
 'bangladesh',
 'brazil',
 'cambodia',
 'canada',
 'chad',
 'che',
 'chile',
 'china',
 'colombia',
 'costa rica',
 'cuba',
 'curaçao',
 'czechia',
 'denmark',
 'dominica',
 'dominican republic',
 'ecuador',
 'el salvador',
 'france',
 'germany',
 'ghana',
 'guatemala',
 'honduras',
 'hong kong',
 'hongkong',
 'india',
 'indonesia',
 'is',
 'israel',
 'italy',
 'japan',
 'lao',
 'ma',
 'malaysia',
 'mali',
 'mauritius',
 'mexico',
 'mongolia',
 'mozambique',
 'myanmar',
 'n/a',
 'nepal',
 'netherlands',
 'nicaragua',
 'nigeria',
 'oman',
 'pakistan',
 'panama',
 'paraguay',
 'peru',
 'philippines',
 'poland',
 'puerto rico',
 'singapore',
 'somalia',
 'south africa',
 'spain',
 'sri lanka',
 'thailand',
 'turkey',
 'uganda',
 'ukraine',
 'united kingdom',
 'viet nam',
 'vietnam',
 'zimbabwe']

In [58]:
# drop not a country
df_country_group = df_country_group.drop(['n/a', 'ai', 'is', 'ma', 'che'])
df_country_group

,time_text,author.name,message,pycountry_name,pycountry_nospace,pycountry_flag,country_re,country_re2,country_re3,country_nltk_2,country_nltk_3
country_vote,,,,,,,,,,,
angola,10,10,10,10,10,10,10,10,10,10,10
argentina,3,3,3,3,3,3,3,3,3,3,3
australia,10,10,10,10,10,10,10,10,10,10,10
bangladesh,2,2,2,2,2,2,2,2,2,2,2
brazil,4360,4360,4360,4360,4360,4360,4360,4360,4360,4360,4360
...,...,...,...,...,...,...,...,...,...,...,...
ukraine,24,24,24,24,24,24,24,24,24,24,24
united kingdom,2,2,2,2,2,2,2,2,2,2,2
viet nam,1398,1398,1398,1398,1398,1398,1398,1398,1398,1398,1398


In [59]:
# change viet nam => vietnam
df_country_group = df_country_group.rename(index={'viet nam': 'vietnam'})
df_country_group

,time_text,author.name,message,pycountry_name,pycountry_nospace,pycountry_flag,country_re,country_re2,country_re3,country_nltk_2,country_nltk_3
country_vote,,,,,,,,,,,
angola,10,10,10,10,10,10,10,10,10,10,10
argentina,3,3,3,3,3,3,3,3,3,3,3
australia,10,10,10,10,10,10,10,10,10,10,10
bangladesh,2,2,2,2,2,2,2,2,2,2,2
brazil,4360,4360,4360,4360,4360,4360,4360,4360,4360,4360,4360
...,...,...,...,...,...,...,...,...,...,...,...
ukraine,24,24,24,24,24,24,24,24,24,24,24
united kingdom,2,2,2,2,2,2,2,2,2,2,2
vietnam,1398,1398,1398,1398,1398,1398,1398,1398,1398,1398,1398


In [65]:
# group by country
df_country_group = df_country_group.groupby(['country_vote']).sum()
df_country_group

,time_text,author.name,message,pycountry_name,pycountry_nospace,pycountry_flag,country_re,country_re2,country_re3,country_nltk_2,country_nltk_3
country_vote,,,,,,,,,,,
angola,10,10,10,10,10,10,10,10,10,10,10
argentina,3,3,3,3,3,3,3,3,3,3,3
australia,10,10,10,10,10,10,10,10,10,10,10
bangladesh,2,2,2,2,2,2,2,2,2,2,2
brazil,4360,4360,4360,4360,4360,4360,4360,4360,4360,4360,4360
...,...,...,...,...,...,...,...,...,...,...,...
uganda,23,23,23,23,23,23,23,23,23,23,23
ukraine,24,24,24,24,24,24,24,24,24,24,24
united kingdom,2,2,2,2,2,2,2,2,2,2,2


In [66]:
df_country_group.index

Index(['angola', 'argentina', 'australia', 'bangladesh', 'brazil', 'cambodia',
       'canada', 'chad', 'chile', 'china', 'colombia', 'costa rica', 'cuba',
       'curaçao', 'czechia', 'denmark', 'dominica', 'dominican republic',
       'ecuador', 'el salvador', 'france', 'germany', 'ghana', 'guatemala',
       'honduras', 'hong kong', 'hongkong', 'india', 'indonesia', 'israel',
       'italy', 'japan', 'lao', 'malaysia', 'mali', 'mauritius', 'mexico',
       'mongolia', 'mozambique', 'myanmar', 'nepal', 'netherlands',
       'nicaragua', 'nigeria', 'oman', 'pakistan', 'panama', 'paraguay',
       'peru', 'philippines', 'poland', 'puerto rico', 'singapore', 'somalia',
       'south africa', 'spain', 'sri lanka', 'thailand', 'turkey', 'uganda',
       'ukraine', 'united kingdom', 'vietnam', 'zimbabwe'],
      dtype='object', name='country_vote')

In [67]:
# summary of country and sort by number of messages
df_country_summary = df_country_group[['message']].sort_values(by=['message'], ascending=False)
# change column name
df_country_summary = df_country_summary.rename(columns={'message': 'amount'}).reset_index()
df_country_summary


,country_vote,amount
0,thailand,12476
1,cambodia,5266
2,brazil,4360
3,indonesia,3723
4,philippines,1807
...,...,...
59,czechia,1
60,oman,1
61,denmark,1
62,mali,1


In [68]:
df_country_summary.to_csv('country_summary.csv', index=False)

In [113]:
# replace Hongkong => Hong Kong
df_country_summary = df_country_summary.replace('hongkong', 'Hong Kong')
df_country_summary = df_country_summary.replace('lao', 'Laos')
df_country_summary = df_country_summary.replace('curaçao', 'Curacao')
df_country_summary = df_country_summary.replace('czechia', 'Czech Republic')

In [114]:
df_country_group.index

Index(['angola', 'argentina', 'australia', 'bangladesh', 'brazil', 'cambodia',
       'canada', 'chad', 'chile', 'china', 'colombia', 'costa rica', 'cuba',
       'curaçao', 'czechia', 'denmark', 'dominica', 'dominican republic',
       'ecuador', 'el salvador', 'france', 'germany', 'ghana', 'guatemala',
       'honduras', 'hong kong', 'hongkong', 'india', 'indonesia', 'israel',
       'italy', 'japan', 'lao', 'malaysia', 'mali', 'mauritius', 'mexico',
       'mongolia', 'mozambique', 'myanmar', 'nepal', 'netherlands',
       'nicaragua', 'nigeria', 'oman', 'pakistan', 'panama', 'paraguay',
       'peru', 'philippines', 'poland', 'puerto rico', 'singapore', 'somalia',
       'south africa', 'spain', 'sri lanka', 'thailand', 'turkey', 'uganda',
       'ukraine', 'united kingdom', 'vietnam', 'zimbabwe'],
      dtype='object', name='country_vote')

In [115]:
# Import the AWOC package.
import awoc
# Initialize the AWOC class.
my_world = awoc.AWOC()

# get countinent from country
df_country_summary['continent'] = df_country_summary['country_vote'].apply(lambda x: my_world.get_country_continent_name(x))
df_country_summary

,country_vote,amount,continent
0,thailand,12476,Asia
1,cambodia,5266,Asia
2,brazil,4360,South America
3,indonesia,3723,Asia
4,philippines,1807,Asia
...,...,...,...
59,Czech Republic,1,Europe
60,oman,1,Asia
61,denmark,1,Europe
62,mali,1,Africa


In [123]:
# rename country_vote to country
df_country_summary = df_country_summary.rename(columns={'country_vote': 'country'})

# change to Upper case
df_country_summary['continent'] = df_country_summary['continent'].str.upper()
df_country_summary['country'] = df_country_summary['country'].str.upper()
df_country_summary

,country,amount,continent
0,THAILAND,12476,ASIA
1,CAMBODIA,5266,ASIA
2,BRAZIL,4360,SOUTH AMERICA
3,INDONESIA,3723,ASIA
4,PHILIPPINES,1807,ASIA
...,...,...,...
59,CZECH REPUBLIC,1,EUROPE
60,OMAN,1,ASIA
61,DENMARK,1,EUROPE
62,MALI,1,AFRICA


In [124]:
df_country_summary.to_csv('country_summary_final.csv', index=False)